# Cython

* Cython is one of python's *dialects* to bridge between C and python.

* Following code block is an [example](https://stackoverflow.com/questions/35656604/running-cython-in-jupyter-ipython) of the cython code.

In [ ]:
%load_ext Cython
# make cython feature avaiable in ipython 


In [ ]:
%%cython --annotate
# with command above, ipython would regard this cell as cython
# cell magic command "%%" must be on the first line of the cell
# --annotate option would present c code from the cython code
# later click on the + signs of the yellow highlight lines


# declare a cython function
def geo_prog_cython(double alpha, int n):
    """
    Sum of a geographic sequence
    Cython version

    ==========
    Parameters
    ==========

    double alpha : ratio
    int n : number of terms
    """
    
    # local variables for this function 
    # with types and initial values
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    
    # accumulation loop
    for i in range(n):
        # current term of a geometric sequence
        current *= alpha
        # accumulation
        sum += current

    # sum of the geometric sequence    
    return sum



* As you can see, cython version has type information.

* Let's try calling the cython function

In [ ]:
%%timeit
# the command above would measure time of this cell
# repeating if necessary

geo_prog_cython(0.5, 5)



* To compare the computation time, let's prepare a python version.

In [ ]:
def geo_prog_python(alpha, n):
    """
    Sum of a geographic sequence
    Python version

    ==========
    Parameters
    ==========

    alpha : ratio
    n : number of terms
    """    

    # initialize
    current = 1.0
    sum = current
    
    # accumulation loop
    for i in range(n):
        current *= alpha
        # accumulation
        sum += current
    return sum



In [ ]:
%%timeit

geo_prog_python(0.5, 5)



* Can you see the benefit?

* Here `%%cython` command is doing all the plumbing for us.
* Under the hood, more is going on as we may see later.

## Visualizing

* `matplotlib` can visualize results from cython functions.

* Following code presents Euler's method simulation.

In [ ]:
%load_ext Cython
# Repeated to help locating


In [ ]:
# simulation parameters
t_start_sec = 0
t_end_sec = 10
delta_t_sec = 1e-3



In [ ]:
%%cython --annotate
# with command above, ipython would regard this cell as cython
# cell magic command "%%" must be on the first line of the cell
# --annotate option would present c code from the cython code

# https://stackoverflow.com/questions/29036068/how-to-expose-a-numpy-array-from-c-array-in-cython
cimport numpy as np
# for cimport, please see:
# https://stackoverflow.com/questions/29311207/cython-compilation-import-vs-cimport
# it would make C API available for np.ndarray[] later
import numpy as np
# for np.empty in this case


def diff_eq(double xi):
    """
    Differential equation to solve
    x' + x = 0
    x' = - x
    """

    # dx/dt + x = 0
    # return value == result of this function
    return -xi


def euler_cython_numpy_array(double ti_sec=0.0, double te_sec=10, double delta_t_sec=1e-3):
    """
    Cython + Numpy Simulation
    Euler's method
    
    ===============
    Input arguments
    ===============
    ti_sec : start time
    te_sec : end time
    delta_t_sec : time step
    """

    # initial values
    # inital value
    cdef double x0 = 1.0
    
    # time variable
    cdef double t = ti_sec
    
    # number of time steps
    cdef int n = int((te_sec - ti_sec) / delta_t_sec) + 1
    # time step
    cdef int i = 0
    
    # https://stackoverflow.com/questions/25974975/cython-c-array-initialization
    # simulation time
    cdef np.ndarray [double, ndim=1, mode="c"] result_t = np.empty(n)
    # state variable 
    cdef np.ndarray [double, ndim=1, mode="c"] result_x = np.empty(n)
    
    # local variable for x'
    cdef double dx_dt = 0
    
    # initial values of time and state
    result_t[0] = ti_sec
    result_x[0] = x0
    
    # time step loop
    for i in range(1, n):
        # slope
        dx_dt = diff_eq(result_x[i-1])
        # state variable of next time step
        result_x[i] = result_x[i-1] + dx_dt * delta_t_sec
        # time step
        result_t[i] = result_t[i-1] + delta_t_sec

    # result of simulation
    return result_t, result_x



* To compare, following cell prepares an equivalent simulation in python.

In [ ]:
def diff_eq_python(xi):

    # dx/dt + x = 0
    # dx/dt = - x
    return -xi


def euler_python(ti_sec = 0.0, te_sec = 10.0, delta_t_sec = 1e-3):

    # initial value
    x0 = 1.0
    
    # number of time steps
    n = int((te_sec - ti_sec) / delta_t_sec) + 1
    
    # prepare for buffers before the loop
    result_t = [0.0] * n
    result_x = [0.0] * n
    
    # set initial value
    result_t[0] = ti_sec
    result_x[0] = x0
    
    # time step loop
    for i in range(1, n):
        dx_dt = diff_eq_python(result_x[i-1])
        result_x[i] = result_x[i-1] + dx_dt * delta_t_sec
        result_t[i] = result_t[i-1] + delta_t_sec

    return result_t, result_x



* Let's run both cython and python versions

In [ ]:
%%time
# measure time to calculate

t_cy_np, x_cy_np = euler_cython_numpy_array(
    t_start_sec,
    t_end_sec,
    delta_t_sec,
)



In [ ]:
%%time
# measure time to calculate

t_py, x_py = euler_python(
    t_start_sec,
    t_end_sec,
    delta_t_sec,
)



* Let's plot both results

In [ ]:
# to plot properly, may need to separate this line
import pylab as py



In [ ]:
py.plot(t_cy_np, x_cy_np, 'o', label='cython numpy')
py.plot(t_py, x_py, '-', label='python')

py.grid(True)
py.xlabel('t')
py.ylabel('y')
py.legend(loc=0)

py.show()



In [ ]:
# measure time to calculate
# -o makes the result available as output

time_cython_dynamic_numpy = %timeit -o t_np, x_np = euler_cython_numpy_array(t_start_sec, t_end_sec, delta_t_sec,)



In [ ]:
# %timeit is a magic command to measure time
# -o makes the result available as output
time_python = %timeit -o t_py, x_py = euler_python(t_start_sec, t_end_sec, delta_t_sec,)



## Calling C/C++ functions

* Cython can call C/C++ functions as follows.
[[ref0](https://cython.readthedocs.io/en/latest/src/userguide/external_C_code.html)]
, [[ref1](https://stackoverflow.com/questions/37426534/how-can-i-import-an-external-c-function-into-an-ipython-notebook-using-cython)]
, [[ref2](https://stackoverflow.com/questions/19260253/cython-compiling-error-multiple-definition-of-functions)]
, [[ref3](https://media.readthedocs.org/pdf/cython/stable/cython.pdf)]
, [[ref4](http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html)]
 
 

### With `numpy` support

* If we need to use matrices and vectors frequently, combining `numpy` and cython may be helpful.

* Let's take a look at an example of calculating cosine values.

* Following is the header file for the C source code.

In [ ]:
%%writefile cos_cython_numpy.h
/*
    2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/
void cos_cython_numpy_c_func(double * in_array, double * out_array, int size);



* Following is the source code file to wrap with cython.

In [ ]:
%%writefile cos_cython_numpy.c
/*
2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/

#include <math.h>

/*  Compute the cosine of each element in in_array, storing the result in
 *  out_array. */
void cos_cython_numpy_c_func(double * in_array, double * out_array, int size){
    int i;
    for(i=0;i<size;i++){
        out_array[i] = cos(in_array[i]);
    }
}



* Following is the cython `.pyx` file calling the C function.

In [ ]:
%%writefile _cos_cython_numpy.pyx
""" Example of wrapping a C function that takes C double arrays as input using
    the Numpy declarations from Cython 
    Valentin Haenel, 2.8.5.2. Numpy Support, 2.8.5. Cython, Scipy Lectures, Oct 18 2016, [Online] 
        Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13 
"""

""" Example of wrapping a C function that takes C double arrays as input using
    the Numpy declarations from Cython """

# cimport the Cython declarations for numpy
cimport numpy as np

# if you want to use the Numpy-C-API from Cython
# (not strictly necessary for this example, but good practice)
np.import_array()

# cdefine the signature of our c function
cdef extern from "cos_cython_numpy.h":
    void cos_cython_numpy_c_func (double * in_array, double * out_array, int size)

# create the wrapper code, with numpy type annotations
def cos_cython_numpy_py_func(np.ndarray[double, ndim=1, mode="c"] in_array not None,
                     np.ndarray[double, ndim=1, mode="c"] out_array not None):
    cos_cython_numpy_c_func(<double*> np.PyArray_DATA(in_array),
                <double*> np.PyArray_DATA(out_array),
                in_array.shape[0])



* Following is file `setup.py`.  Running this `.py` file would build the C function wrapper.

In [ ]:
%%writefile setup.py

# Valentin Haenel, 2.8. Interfacing with C,  Scipy Lectures, Oct 18 2016, [Online]
#   Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html

from distutils.core import setup, Extension
# distutils : building and installing modules 
# https://docs.python.org/3/library/distutils.html

import numpy
from Cython.Distutils import build_ext

print('for NumPy Support of Cython '.ljust(60, '#'))
setup(cmdclass={'build_ext': build_ext},
      ext_modules=[Extension("cos_cython_numpy",
                             sources=['_cos_cython_numpy.pyx', "cos_cython_numpy.c"],
                             include_dirs=[numpy.get_include()])],
      )



In [ ]:
%%bash

# this is the command to build C function wrapper
python setup.py build_ext --inplace
# with --inplace, the module would be in the same folder.



* Now let's import the module with c wrapper function

In [ ]:
# the cython module including C function
import cos_cython_numpy

# to visualize the result
import pylab as py

# allocate arrays externally
x = py.arange(0, 2 * py.pi, 0.1)
y = py.empty_like(x)

# c wrapper function
cos_cython_numpy.cos_cython_numpy_py_func(x, y)

# plot the result
py.plot(x, py.cos(x), 'o', label='cos numpy')
py.plot(x, y, '.', label='cos cython')
py.show()



* Cleanup

In [ ]:
%%bash
rm cos_cython_numpy.h
rm cos_cython_numpy.c
rm _cos_cython_numpy.c
rm _cos_cython_numpy.pyx
rm setup.py



* Check results

In [ ]:
cos = py.cos(x)

for i in range(len(cos)):
    assert y[i] == cos[i], "Cython Result != Pylab Expected"



### Revisiting Euler's method

* Let's try wrapping a C/C++ version of the Euler's method above.

* Following is the header file for the C source code.

In [ ]:
%%writefile euler_cython_numpy.h
/*
    2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/
void euler_cython_c_function(
        const double t_start_sec, const double t_end_sec,
        const double delta_t_sec,
        const int size,
        const double * result_t, const double * result_x
    );



* Following is a C source code example.

In [ ]:
%%writefile euler_cython_numpy.c
#include    <assert.h>


double diff_eq(double xi){
    return -xi;
}


void euler_cython_c_function(
        const double t_start_sec,
        const double t_end_sec,
        const double delta_t_sec,
        const int size,
        double * result_t, double * result_x
    ){
    // Simulation start and end time

    // Initial state
    double x0 = 1.0;
    double dx_dt = 0.0;
    
    int i = 0;
    
    // Length of simulation
    const int n = (int) ((t_end_sec - t_start_sec) / delta_t_sec) + 1;

    // Check array size
    assert(size > n);

    // Set initial value    
    result_t[0] = t_start_sec;
    result_x[0] = x0;
    
    // Time step loop
    // Watch the last value of i here
    for (i=0; (n-1)>i; ++i){
        // Calculate derivative
        dx_dt = diff_eq(result_x[i]);

        // Calculate state value of the next step
        result_x[i+1] = result_x[i] + dx_dt * delta_t_sec;

        // Calculate time of next step
        result_t[i+1] = result_t[i] + delta_t_sec;
    }

    return;
}



* Following is file `_euler_cython_numpy.pyx`.

In [ ]:
%%writefile _euler_cython_numpy.pyx
# cimport the Cython declarations for numpy
cimport numpy as np
# to dynamically allocate numpy.ndarray using np.empty()
import numpy as np

# Recommended practice for Numpy-C-API from Cython
np.import_array()

# cdefine the signature of our c function
cdef extern from "euler_cython_numpy.h":
    void euler_cython_c_function (
        const double t_start_sec,
        const double t_end_sec,
        const double delta_t_sec,
        const int size,
        double * result_t, double * result_x,
    )
# now we can call euler_cython_c_function()

# create the wrapper code, with numpy type annotations
# a python program would call this function
# and this function would call the C function
def euler_cython_numpy_py_func(
        double t_start_sec=0,
        double t_end_sec=10,
        double delta_t_sec=1e-3,
    ):

    # number of time steps
    cdef int n = int((t_end_sec - t_start_sec) / delta_t_sec) + 1
    
    # preallocate output arrays
    cdef np.ndarray[double, ndim=1, mode="c"] t_array = np.empty(n)
    cdef np.ndarray[double, ndim=1, mode="c"] x_array = np.empty(n)

    # calling C function
    euler_cython_c_function(
        t_start_sec,
        t_end_sec,
        delta_t_sec,
        t_array.shape[0],
        <double*> np.PyArray_DATA(t_array),
        <double*> np.PyArray_DATA(x_array),
    )
    # <double*> np.PyArray_DATA(t_array)
    # this line would obtain a C double pointer for t_array
    # Cython MemoryView can be an alternative
    # https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html

    return t_array, x_array



* Following is file `setup.py`.

In [ ]:
%%writefile setup.py

# Valentin Haenel, 2.8. Interfacing with C,  Scipy Lectures, Oct 18 2016, [Online]
#   Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html

from distutils.core import setup, Extension

import numpy
from Cython.Distutils import build_ext

print('for NumPy Support of Cython '.ljust(60, '#'))
setup(cmdclass={'build_ext': build_ext},
      ext_modules=[
                  Extension(
                        "euler_cython_numpy", 
                        sources=['_euler_cython_numpy.pyx', "euler_cython_numpy.c"], 
                        include_dirs=[numpy.get_include()]
                  ),
            ],
      )



* Now build the module.

In [ ]:
%%bash
python setup.py build_ext --inplace



* Let's import the cython c wrap module, run the simulation, and plot the results.

In [ ]:
import pylab as py
import euler_cython_numpy as cwrap

# call the simulation function
t_cy_wrap, x_cy_wrap = cwrap.euler_cython_numpy_py_func(
    t_start_sec,
    t_end_sec,
    delta_t_sec,
)

# plot result from python simulation
py.plot(t_py, x_py, 'o', label='python')

# plot result from cython + numpy
py.plot(t_cy_np, x_cy_np, '.', label='cython numpy')

# plot result from cython c wrapper function
py.plot(t_cy_wrap, x_cy_wrap, '-', label='cython wrap')

py.legend(loc=0)
py.grid(True)
py.show()



* Let's compare the calculation time again.

In [ ]:
# cython wrap with memory allocation
# -o makes the result available as output
time_c_wrap_malloc = %timeit -o t_cy_wrap, x_cy_wrap = cwrap.euler_cython_numpy_py_func(t_start_sec, t_end_sec, delta_t_sec,)



* Clean up the files.

In [ ]:
%%bash
rm euler_cython_numpy.h
rm euler_cython_numpy.c
rm _euler_cython_numpy.c
rm _euler_cython_numpy.pyx
rm setup.py



* Check array type and dimension size

In [ ]:
assert isinstance(t_cy_wrap, py.ndarray), "t is not a numpy.ndarray"
assert isinstance(x_cy_wrap, py.ndarray), "x is not a numpy.ndarray"

# for now expect 1-dimensional arrays
assert 1 == t_cy_wrap.ndim, "t dimension > 1"
assert 1 == x_cy_wrap.ndim, "x dimension > 1"



* Check each element of the results

In [ ]:
for i in range(len(t_py)):
    assert t_cy_np[i] == t_py[i], f"t[{i}] Cython NumPy Result f{t_cy_np[i]} != Python Result f{t_py[i]}\n"
    assert x_cy_np[i] == x_py[i], f"x[{i}] Cython NumPy Result f{x_cy_np[i]} != Python Result f{x_py[i]}\n"
    assert t_cy_wrap[i] == t_py[i], f"t[{i}] Cython C Wrap Result f{t_cy_wrap[i]} != Python Result f{t_py[i]}\n"
    assert x_cy_wrap[i] == x_py[i], f"x[{i}] Cython C Wrap Result f{x_cy_wrap[i]} != Python Result f{x_py[i]}\n"

assert abs(t_py[-1] - t_end_sec) < (0.5 * delta_t_sec), f"t_end_sec = {t_end_sec}, t_py[-1] = {t_py[-1]}"
assert abs(t_cy_np[-1] - t_end_sec) < (0.5 * delta_t_sec), f"t_end_sec = {t_end_sec}, t_cy_np[-1] = {t_cy_np[-1]}"
assert abs(t_cy_wrap[-1] - t_end_sec) < (0.5 * delta_t_sec), f"t_end_sec = {t_end_sec}, t_cy_wrap[-1] = {t_cy_wrap[-1]}"



* Let's try different simulation time

In [ ]:
import pylab as py
import euler_cython_numpy as cwrap


t_start_2_sec = 0.0
t_end_2_sec   = 1.0
delta_t_2_sec = 2.0 ** (-10)

# cython numpy
t_2_np, x_2_np = euler_cython_numpy_array(
    t_start_2_sec, t_end_2_sec, delta_t_2_sec,
)

# python list
t_2_py, x_2_py = euler_python(
    t_start_2_sec, t_end_2_sec, delta_t_2_sec,
)

# cython C wrap
t_2_cy_wrap, x_2_cy_wrap = cwrap.euler_cython_numpy_py_func(
    t_start_2_sec, t_end_2_sec, delta_t_2_sec,
)

# plots
py.plot(t_2_py, x_2_py, 'o', label='python list')
py.plot(t_2_np, x_2_np, '.', label='cython numpy')
py.plot(t_2_cy_wrap, x_2_cy_wrap, '-', label='cython wrap')

py.legend(loc=0)
py.grid(True)
py.show()



* Let's check if cython numpy result correct

In [ ]:
for i in range(len(t_2_py)):
    assert t_2_np[i] == t_2_py[i], f"t[{i}] Cython NumPy Result f{t_2_np[i]} != Python Result f{t_2_py[i]}\n"
    assert x_2_np[i] == x_2_py[i], f"x[{i}] Cython NumPy Result f{x_2_np[i]} != Python Result f{x_2_py[i]}\n"
    assert t_2_cy_wrap[i] == t_2_py[i], f"t[{i}] Cython C Wrap Result f{t_2_cy_wrap[i]} != Python Result f{t_2_py[i]}\n"
    assert x_2_cy_wrap[i] == x_2_py[i], f"x[{i}] Cython C Wrap Result f{x_2_cy_wrap[i]} != Python Result f{x_2_py[i]}\n"

assert abs(t_2_py[-1] - t_end_2_sec) < (0.5 * delta_t_2_sec), f"t_end_2_sec = {t_end_2_sec}, t_2_py[-1] = {t_2_py[-1]}"
assert abs(t_2_np[-1] - t_end_2_sec) < (0.5 * delta_t_2_sec), f"t_end_2_sec = {t_end_2_sec}, t_2_np[-1] = {t_2_np[-1]}"
assert abs(t_2_cy_wrap[-1] - t_end_2_sec) < (0.5 * delta_t_2_sec), f"t_end_2_sec = {t_end_2_sec}, t_2_cy_wrap[-1] = {t_2_cy_wrap[-1]}"



* Let's compare the computation times of different versions:

In [ ]:
import IPython.display as disp

disp.display(
    disp.Markdown(
        '\n'.join([
            "| type | result |",
            "|:-----:|:-----:|",
            f"| cython c wrap (with memory allocation) | {time_c_wrap_malloc} |",
            f"| cython dynamic NumPy | {time_cython_dynamic_numpy} |",
            f"| python list | {time_python} |",
            ]
        )
    )
)

## Python Control Open Source Project

* Nowadays in late 2010s, there is an open source project of [**python-control**](https://github.com/python-control/).

* The objective is to develop a useful tool for control engineers in python.

* One of the challenges is a numerical library named [**Slycot**](https://github.com/python-control/Slycot) mostly in Fortran.

* Slycot uses other numerical libraries such as **BLAS** and **LAPACK** also in Fortran.

* It would be noteworthy that `numpy` of SciPy Stack also includes **BLAS** and **LAPACK**.

* Please refer to [**python-control**](https://github.com/python-control/python-control) and [**Slycot**](https://github.com/python-control/Slycot) for more information.

## Exercise

### 00 Allocation in `.pyx`

* In the cosine example, see if you can pre-allocate output array in `.pyx` file.

### 01 Other numerical methods

* From the Euler's method example above, implement other numerical methods and compare the performance. 